In [3]:
import openai
from dotenv import load_dotenv

load_dotenv()

True

In [34]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-7ahBpDhjtiatPsnKvZkEap6fs0d7V at 0x7fb968815860> JSON: {
  "id": "chatcmpl-7ahBpDhjtiatPsnKvZkEap6fs0d7V",
  "object": "chat.completion",
  "created": 1688980185,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The World Series in 2020 was played at Globe Life Field in Arlington, Texas."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 53,
    "completion_tokens": 18,
    "total_tokens": 71
  }
}